In [1]:
from __future__ import division

%matplotlib inline

#Data crunch and visualize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#Data formats
import csv
from collections import defaultdict
import re

#Modeling
import statsmodels.api as sm
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

#SQL
# from sqlalchemy import MetaData
# from sqlalchemy import create_engine
# from sqlalchemy.engine.url import URL

#Gen
#import settings
import time
import glob
import os
from datetime import datetime, timedelta
import traceback
import pickle
import scipy.io as sio
import scipy
import tqdm

In [2]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, precision_recall_curve, roc_curve, roc_auc_score
# from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import Imputer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn import feature_selection 

# import xgboost
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, precision_recall_curve, roc_curve, roc_auc_score
# from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.cross_validation import cross_val_score
# from sklearn.grid_search import GridSearchCV

In [3]:
pd.set_option("display.max_rows",300)
pd.set_option("display.max_columns",300)
np.set_printoptions(threshold=1000)
pd.set_option('display.date_yearfirst', True)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
import matplotlib.dates as mdates

import matplotlib
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 14}

matplotlib.rc('font', **font)

In [5]:
ls ../../../work/InternalMedicine/s176036/Anemia/final_data

req1512_CCI_detail_2010.csv*    req1512_lab_2008.csv   req1512_lab_2015.csv
req1512_CCI_summary_2010.csv*   req1512_lab_2009.csv   req1512_lab_2016.csv
req1512_demographics.csv*       req1512_lab_2010.csv*  req1512_lab_2017.csv
req1512_ENC_DX_beyond10dx.csv*  req1512_lab_2011.csv   req1512_lab_2018.csv
req1512_encounter.csv*          req1512_lab_2012.csv   req1512_lab_2019.csv
req1512_lab_2006.csv            req1512_lab_2013.csv   req1512_lab_2020.csv*
req1512_lab_2007.csv            req1512_lab_2014.csv   req1512_med_data.csv*


# Demographics

In [7]:
dem = pd.read_csv('../../../work/InternalMedicine/s176036/Anemia/final_data/req1512_demographics.csv')

In [8]:
# Removes extra spaces from column names for easier future access
dem.columns = [x.strip().lower() for x in dem.columns]

In [9]:
dem.birth_date = pd.to_datetime(dem.birth_date)
dem.death_date = pd.to_datetime(dem.death_date)

In [14]:
len(set(dem.mrn))

509807

# Encounters

In [10]:
enc = pd.read_csv('../../../work/InternalMedicine/s176036/Anemia/final_data/req1512_encounter.csv')

/home2/s176036/.conda/envs/py36-test/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
enc.columns = [x.strip().lower() for x in enc.columns]

In [13]:
enc.contact_date = pd.to_datetime(enc.contact_date)

In [14]:
# Remove Hospital Encounters
enc = enc[enc.enc_type != 'Hospital Encounter']

# Labs


## Exploration

### Load all labs

In [15]:
lab_all = pd.DataFrame()
for year in np.arange(2009, 2021):
    print(str(year))
    lab_cur =pd.read_csv('../../../work/InternalMedicine/s176036/Anemia/final_data/req1512_lab_' + str(year)+'.csv',  error_bad_lines=False)
    lab_cur.columns = [x.strip().lower() for x in lab_cur.columns]
#     lab_cur = lab_cur[lab_cur.common_name == lab_name]
    lab_cur.result_date = pd.to_datetime(lab_cur.result_date)
    lab_cur.ordering_date = pd.to_datetime(lab_cur.ordering_date)
    lab_all = lab_all.append(lab_cur)
    print(str(year) + " deleted")
    del lab_cur

2009


/home2/s176036/.conda/envs/py36-test/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (22,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2009 deleted
2010


/home2/s176036/.conda/envs/py36-test/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (21,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2010 deleted
2011


...

2012 deleted
2013
2013 deleted
2014
2014 deleted
2015
2015 deleted
2016
2016 deleted
2017
2017 deleted
2018
2018 deleted
2019


b'Skipping line 1835963: expected 39 fields, saw 40\n'
b'Skipping line 2854966: expected 39 fields, saw 40\n'
b'Skipping line 2936398: expected 39 fields, saw 40\n'
b'Skipping line 3426864: expected 39 fields, saw 40\n'
b'Skipping line 3528034: expected 39 fields, saw 40\n'
b'Skipping line 4197661: expected 39 fields, saw 40\n'
b'Skipping line 4857875: expected 39 fields, saw 40\n'
b'Skipping line 5496388: expected 39 fields, saw 40\n'
b'Skipping line 5950186: expected 39 fields, saw 40\n'
b'Skipping line 7605266: expected 39 fields, saw 40\n'
b'Skipping line 8033330: expected 39 fields, saw 40\n'
b'Skipping line 8447064: expected 39 fields, saw 40\n'
b'Skipping line 8759073: expected 39 fields, saw 41\n'
b'Skipping line 9507719: expected 39 fields, saw 40\n'
b'Skipping line 11010034: expected 39 fields, saw 40\n'
b'Skipping line 11608262: expected 39 fields, saw 41\n'
b'Skipping line 11867144: expected 39 fields, saw 40\n'
b'Skipping line 12699714: expected 39 fields, saw 40\n'
b'Skip

2019 deleted
2020


b'Skipping line 1026118: expected 39 fields, saw 40\n'
b'Skipping line 1237673: expected 39 fields, saw 42\n'
b'Skipping line 1425322: expected 39 fields, saw 41\n'
b'Skipping line 1429946: expected 39 fields, saw 40\n'
b'Skipping line 1730257: expected 39 fields, saw 40\n'
b'Skipping line 1975121: expected 39 fields, saw 40\n'
b'Skipping line 2317794: expected 39 fields, saw 40\n'
b'Skipping line 2402724: expected 39 fields, saw 41\n'
b'Skipping line 2616483: expected 39 fields, saw 40\n'
b'Skipping line 3767802: expected 39 fields, saw 41\n'
b'Skipping line 3878752: expected 39 fields, saw 40\n'
b'Skipping line 4149871: expected 39 fields, saw 40\n'
b'Skipping line 4365979: expected 39 fields, saw 40\n'
b'Skipping line 4574085: expected 39 fields, saw 40\n'
b'Skipping line 4957117: expected 39 fields, saw 40\n'
b'Skipping line 5408879: expected 39 fields, saw 40\n'
b'Skipping line 5496207: expected 39 fields, saw 40\n'
b'Skipping line 5580172: expected 39 fields, saw 40\n'
b'Skipping

2020 deleted


In [27]:
lab_all.to_csv('../../../work/InternalMedicine/s176036/Anemia/all_labs_2009_2020.csv')

KeyboardInterrupt: 

### Summary Stats of all common names

In [33]:
componentDataDF = pd.read_csv('../../../work/InternalMedicine/s176036/Anemia/Lab_Crosswalk.csv')
componentDataDF.head()

,First Alphabetical Componenet,Related Components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Comments
0,% SATURATION,NaN,NaN,NaN,NaN,Need to see data
1,% SATURATION (FE),IRON SATURATION,NaN,NaN,NaN,NaN
2,% SATURATION TIBC,NaN,NaN,NaN,NaN,Need to see data
3,25-HYDROXY D TOTAL,"VIT D, 25-HYDROXY","VITAMIN D, 25 OH TOTAL","VITAMIN D, 25-HYDROXY",NaN,NaN
4,A/G RATIO,ALB/GLOBULIN RATIO,ALBUMIN/GLOB SERPL,ALBUMIN/GLOBULIN RATIO,NaN,NaN


In [34]:
componentDataDF.rename({'First Alphabetical Componenet':'First Alphabetical Component', 'Related Components':'Related Components 0', 'Unnamed: 2':'Related Components 1', 'Unnamed: 3':'Related Components 2', 'Unnamed: 4':'Related Components 3'}, axis = 1, inplace =True)

In [38]:
listOfLabComponents = []
for eachIndex, eachRow in componentDataDF.iterrows():
    listOfLabComponents.append(eachRow['First Alphabetical Component'])
    for eachColumnName in ['Related Components 0', 'Related Components 1', 'Related Components 2', 'Related Components 3']:
        if isinstance(eachRow[eachColumnName], str):
            listOfLabComponents.append(eachRow[eachColumnName])

print('number of lab components:', len(listOfLabComponents))

number of lab components: 300
PROTIME


In [ ]:
lab_summaries_numeric_only = pd.DataFrame()
for eachComponent in listOfLabComponents:
    print(eachComponent)
    cur = lab_all[(lab_all.common_name == eachComponent) & (lab_all_copy.ord_num_value != 9999999)]
    lab_summaries_numeric_only = lab_summaries_numeric_only.append(pd.DataFrame(cur['ord_num_value'].describe().to_dict(), index = [eachComponent]))

% SATURATION
% SATURATION (FE)
IRON SATURATION
% SATURATION TIBC
25-HYDROXY D TOTAL
VIT D, 25-HYDROXY
VITAMIN D, 25 OH TOTAL
VITAMIN D, 25-HYDROXY
A/G RATIO
ALB/GLOBULIN RATIO
ALBUMIN/GLOB SERPL
ALBUMIN/GLOBULIN RATIO
ABSOLUTE LYMPHOCYTES
LYMPHOCYTE
LYMPHS
ABSOLUTE PLASMA CELLS
PLASMA CELL ABS
ABSOLUTE REACTIVE LYMPHOCYTES
LYMPHS REACTIVE
REAC LYMPHS ABS
ACANTHOCYTES
ACANTHOCYTES NUMERIC
ALBUMIN
ALK PHOS
ALK PHOSPHATASE
ALKALINE PHOSPHATASE
ALKALINE PHOSPHATASE, S
OSH ALP SERUM
ALT
ALT (SGPT)
OSH ALT SERUM
AMIKACIN TR
AMYLASE
ANION GAP
ANISOCYTOSIS
AST
OSH AST SERUM
B-TYPE NATRIURETIC PEPTIDE
BNP
BANDS
BANDS ABS
BANDS PCT
BASO (%) AUTO
BASO%(AUTO)
BASOPHILS, %
BASOS PCT
BASO # AUTO
BASO#(AUTO)
BASOPHILS # BLD AUTO
BASOS ABS
BASO STIPPLING
BASOPHIL
BILIRUBIN
BILIRUBIN, TOTAL
OSH TBIL SERUM
TOTAL BILIRUBIN
BILIRUBIN, DIRECT
BILIRUBIN, INDIRECT
INDIRECT BILIRUBIN
BLAST
BLASTS
BLAST ABS
BLAST CELLS #
BLASTS ABS
BLAST AND ABN LYMPHO
BLASTS/BLAST LIKE CELLS
BLASTS PCT
BLOOD UREA NITROGEN
BUN

In [87]:
lab_summaries = pd.DataFrame()
for eachComponent in listOfLabComponents:
    print(eachComponent)
    cur = lab_all[lab_all.common_name == eachComponent]
    lab_summaries = lab_summaries.append(pd.DataFrame(cur['ord_num_value'].describe().to_dict(), index = [eachComponent]))

% SATURATION
% SATURATION (FE)
IRON SATURATION
% SATURATION TIBC
25-HYDROXY D TOTAL
VIT D, 25-HYDROXY
VITAMIN D, 25 OH TOTAL
VITAMIN D, 25-HYDROXY
A/G RATIO
ALB/GLOBULIN RATIO
ALBUMIN/GLOB SERPL
ALBUMIN/GLOBULIN RATIO
ABSOLUTE LYMPHOCYTES
LYMPHOCYTE
LYMPHS
ABSOLUTE PLASMA CELLS
PLASMA CELL ABS
ABSOLUTE REACTIVE LYMPHOCYTES
LYMPHS REACTIVE
REAC LYMPHS ABS
ACANTHOCYTES
ACANTHOCYTES NUMERIC
ALBUMIN
ALK PHOS
ALK PHOSPHATASE
ALKALINE PHOSPHATASE
ALKALINE PHOSPHATASE, S
OSH ALP SERUM
ALT
ALT (SGPT)
OSH ALT SERUM
AMIKACIN TR
AMYLASE
ANION GAP
ANISOCYTOSIS
AST
OSH AST SERUM
B-TYPE NATRIURETIC PEPTIDE
BNP
BANDS
BANDS ABS
BANDS PCT
BASO (%) AUTO
BASO%(AUTO)
BASOPHILS, %
BASOS PCT
BASO # AUTO
BASO#(AUTO)
BASOPHILS # BLD AUTO
BASOS ABS
BASO STIPPLING
BASOPHIL
BILIRUBIN
BILIRUBIN, TOTAL
OSH TBIL SERUM
TOTAL BILIRUBIN
BILIRUBIN, DIRECT
BILIRUBIN, INDIRECT
INDIRECT BILIRUBIN
BLAST
BLASTS
BLAST ABS
BLAST CELLS #
BLASTS ABS
BLAST AND ABN LYMPHO
BLASTS/BLAST LIKE CELLS
BLASTS PCT
BLOOD UREA NITROGEN
BUN

In [ ]:
lab_summaries.to_csv('../../../work/InternalMedicine/s176036/Anemia/lab_stats_summaries.csv')
lab_summaries.to_csv('../../../work/InternalMedicine/s176036/Anemia/lab_stas_summaries_numeric_only.csv')

## Lab cleaning
* Restrict to non-hospital
* Restrict to >9/2008
* Remove point of care
* Remove if no reference range (note two sources for references)
* Remove external result (30) or external order (143)

In [28]:
import math
        
# NOTE: needs enc from #2 in table of contents where hospital encounters are excluded
def lab_clean(df, use_ref_ranges = True):
    #Restrict to non-hospital through inner join
    df = pd.merge(df, enc['pat_enc_csn_id'], on = 'pat_enc_csn_id', how = 'inner')
    #Restrict to >9/2008
    df = df[df.result_date>datetime(2008, 9,30)]
    #Remove external orders/results
    df = df[(df.order_class_c != 30)&(df.order_class_c != 143)]
    #Remove point of care
    df = df[(~df.proc_name.str.contains('(ABL)|(iStat)|(POC)|(POCT)')) & (df.order_class !='Point of Care')]
    
    #Remove if no ref range if TRUE
    if use_ref_ranges == True:
        df['ref_low_2'] = df.ref_normal_vals.apply(lambda x: x.split('-')[0] if (isinstance(x, str)) else x)
        df['ref_high_2'] = df.ref_normal_vals.apply(lambda x: x.split('-')[1] if ((isinstance(x, str)) and (len(x.split('-'))==2)) else x)
        df.ref_low_2 = pd.to_numeric(df['ref_low_2'], errors='coerce')
        df.ref_high_2 = pd.to_numeric(df['ref_high_2'], errors='coerce')
        df['final_ref_low'] = df.apply(lambda x: x.ref_low_2 if ~np.isnan(x.ref_low_2) else x.reference_low, axis = 1)
        df['final_ref_high'] = df.apply(lambda x: x.ref_high_2 if ~np.isnan(x.ref_high_2) else x.reference_high, axis = 1)
        df = df[(~df.final_ref_low.isna())&(~df.final_ref_high.isna())]
    return df